# Projet TAL M1 S1 : POS Tagging
structure du projet expliqué ici : https://u-paris.zoom.us/rec/share/rFB-gp7vkgMK31kfPxEsyNF2k6q7mHcq3P9MQiovimkr9ebIewZpuD62RBk6SmM.31Mmzx1VyMqQXRtu?startTime=1607605609000

# Implémentation du classifieur

## Import du corpus
Cours Zoom du 17/12 - 35 min

On charge les trois corpus *in-domain* sous la forme de listes de dictionnaires : chaque phrase a une clé "mots" qui est associée à une liste des mots, et une clé "POS" associée à une liste de POS. 

On utilise les trois corpus distincts de French-GSD :
- Train : apprentissage.
- Dev : validation. Pour tester et améliorer le modèle. 
- Test : évaluation. On ne l'utilisera pas pendant l'apprentissage ou le test.

In [1]:
# clés des dictionnaires
mot = "mot"
gold = "gold_label"
mot_prec = "préc"
mot_suiv = "suiv"
maj = "maj"
all_caps = "all caps"
num = "nb"
nonAlphanum = "non alphanum"
long = "long"
court = "court"
un_car = "un caractère"
suff_adv = "suffixe adv"
suff_vb = "suffixe verb"
suff_noun = "suffixe nom"
suff_adj = "suffixe adj"


In [2]:
# j'ai juste rajouté l'encodage car j'avais une erreur sans 

def load_corpus(file):
    with open(file, "r", encoding = "utf8") as f: 
        content = f.read() # chargement du corpus
    content = content.split("\n\n") # séparation en phrases
    corpus = []
    for phrase in content: # pour chaque phrase
        phrase_dico = {"mots" : [], "gold_labels" : [], "lemme" : []} # liste qui contiendra 1 dictionnaire par mot de la phrase
        for line in phrase.splitlines():
            if not line.startswith("#"): # on ignore les lignes qui commencent par #
                features = line.split("\t")
                phrase_dico["mots"].append(features[1])
                phrase_dico["lemme"].append(features[2])
                phrase_dico["gold_labels"].append(features[3])
        corpus.append(phrase_dico)
    return corpus

gsd_train = load_corpus("corpus-in-domain/fr_gsd-ud-train.conllu")
gsd_test = load_corpus("corpus-in-domain/fr_gsd-ud-test.conllu")
gsd_dev = load_corpus("corpus-in-domain/fr_gsd-ud-dev.conllu")

In [3]:
print("---- Aperçus d'une phrase de chaque corpus-----", end="\n\n")
print(gsd_train[1], end="\n\n")
print(gsd_test[102], end="\n\n")
print(gsd_dev[564])

---- Aperçus d'une phrase de chaque corpus-----

{'mots': ["L'", 'œuvre', 'est', 'située', 'dans', 'la', 'galerie', 'des', 'de', 'les', 'batailles', ',', 'dans', 'le', 'château', 'de', 'Versailles', '.'], 'gold_labels': ['DET', 'NOUN', 'AUX', 'VERB', 'ADP', 'DET', 'NOUN', '_', 'ADP', 'DET', 'NOUN', 'PUNCT', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT'], 'lemme': ['le', 'œuvre', 'être', 'situer', 'dans', 'le', 'galerie', '_', 'de', 'le', 'bataille', ',', 'dans', 'le', 'château', 'de', 'Versailles', '.']}

{'mots': ['La', 'gestion', 'et', "l'", 'exploitation', 'de', 'la', 'salle', 'de', 'concert', 'Wagram', ',', 'récemment', 'rénovée', ',', 'sera', 'assurée', 'par', 'Eurosites', ',', 'leader', 'en', 'France', 'de', 'la', 'location', 'de', 'salles', '.'], 'gold_labels': ['DET', 'NOUN', 'CCONJ', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'PROPN', 'PUNCT', 'ADV', 'VERB', 'PUNCT', 'AUX', 'VERB', 'ADP', 'PROPN', 'PUNCT', 'NOUN', 'ADP', 'PROPN', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', '

## Extraction des caractéristiques

La fonction feature_extraction renvoie une liste de dictionnaires (un par mot) qui contiennent les caractéristiques suivantes pour chaque mot :
- mot
- mot précédent : pour le premier mot de la phrase son mot précédent sera "START", ce qui permettra de prendre en compte la caractéristique "être le premier mot".
- mot suivant : pour le dernier mot, ce sera "END".
- commence par une lettre majuscule
- est entièrement en majuscules
- contient des chiffres
- contient des caractères non alphanumériques
- longueur du mot (3 caractéristiques binaires) : a 1 seule caractère, a moins de 3 caractères, a plus de 3 caractères.
- a un suffixe nominal
- a un suffixe adjectival
- a un suffixe verbal (ça marche que sur les lemmes !)
- a un suffixe adverbial (là ça marche sur la forme fléchie)

Met-on dans les listes les suffixes qui appartiennent à 2 catégories ?
Peut-on utiliser les lemmes ?

On pourrait ajouter des suffixes aussi. Par exemple une feature pourrait être "finir en -ment", une autre suffi_nominaux (-al, -ion, etc.), etc. 
On est confrontées à un problème d'homonymie avec certains suffixes. Par exemple, -ains dans certains (pronom) et hautains (adj). A voir comment gérer ça. 

On ne conserve plus la structure des phrases, qui n'est plus nécessaire une fois qu'on a extrait les informations comme mot précédent et mot suivant.

In [8]:
# étape préliminaire : création d'une liste du vocabulaire de notre corpus, utilisée pour créer les vecteurs creux

def getVoc(training_set):
    voc_set = set()
    for phrase in training_set:
        for mot in phrase["mots"]:
            voc_set.add(mot.lower()) # on ajoute les mots sans majuscules
            # les majuscules sont prises en compte dans une autre caractéristique, donc on ne perd pas d'information
    voc_set.add("start")
    voc_set.add("end")

    index = 0
    voc_dico = {} # on transfère le set dans un dictionnaire pour accéder plus rapidement aux indices 
    for word in list(voc_set):
        voc_dico[word] = index
        index +=1

    return voc_dico

gsd_train_voc = getVoc(gsd_train)
print(f"Taille du voc : {len(gsd_train_voc)}", end = "\n\n")

Taille du voc : 39708



In [26]:
print(list(gsd_train_voc.items())[:30])

[('usine', 0), ('doubs', 1), ('jeb', 2), ('doter', 3), ('formèrent', 4), ('théosophique', 5), ('éviter', 6), ('mystérieuse', 7), ('cuba', 8), ('tcheu', 9), ('sparta', 10), ('pratiqué', 11), ('légué', 12), ('pentecôte', 13), ('dubourdieu', 14), ('pilote', 15), ('lords', 16), ('sépey', 17), ('incarnait', 18), ('céramiques', 19), ('adélaïde', 20), ('accossato', 21), ('633 400', 22), ('visegrád', 23), ('habache', 24), ('regroupés', 25), ('amenemhat', 26), ('web', 27), ('fit', 28), ('distal', 29)]


In [9]:
def feature_extraction(corpus, voc):

    corpus_features = []
    """
    Listes à utiliser pour les lemmes :
    list_adj = ["ain", "aine","aire","é", "ée","iel", "uel", "lle",  "al", "ales", "al", "ial","er", "ère", "ier", "esque", "eur", "euse", "ieux","ueux", "if","ive", "in","ine","ique","atoire", "u","ue", "issime", "able","ible", "uble", "ième","uple"]
    list_noun = ["ade", "age","aille", "aison", "ison","oison", "ation", "ition","ssion", "sion","xion", "isation","ment", "ement","erie", "ure","ature","at", "ance","ence", "escence","ité", "eté","té", "ie","erie", "esse","ise", "eur","isme", "iste", "seur", "isseur", "isateur", "euse","isseuse", "atrice","ier","ière", "aire","ien", ienne","iste",er","eron","eronne","trice","oir", "oire","ier","ière","erie","anderie", "aire","ain","aille", "ée", "ard","asse", "assier","âtre","aut","eau","ceau", "ereau","eteau", "elle", "et","elet","ette", 
"elette","in","otin", "ine" "illon","on", "ille", "erole","ole","iche"]
    """

    list_vb = ["iser","ifier", "oyer","ailler", "asser","eler", "eter","iller", "iner","nicher", "ocher","onner","otter","oter", "ouiller"]
    list_adj = ["ain", "aine","ains", "aines","aire", "aires","é", "ée","ées", "és","iel", "iels","uel", "uels", 
"lle", "lles","els", "el" "al", "ales", "al", "ial", "aux","iaux", "er","ers", "ère","ères", "ier", "iers",      "esque","esques", "eur","eurs", "euse","euses", "ieux","ueux", "if", "ifs","ive", "ives","in", "ins","ine",      "ines","iques", "ique","atoire", "u","ue", "us","ues", "issime","issimes","able","ible", "ibles","ables",  
    "uble","ubles", "ième","ièmes", "uple"]
    list_noun = ["ade", "ades", "age", "ages","aille", "ailles", "aison", "ison", "isons","oison", "ation", "itions", "ition", "ssion", "sion","xion", "isation","ment", "ement","erie", "eries","ure","ures","ature", "atures","at", "ance","ence", "escence","ité", "eté","té", "ie","erie", "esse", "ise", "eur","isme", "iste", "istes","eurs", "seur","seurs", "isseur","isseurs", "isateur","euse", "euses","isseuse", "isseuses",       "atrice", "atrices","ier", "iers","ière", "ières","aire","aires","ien", "iens","ienne", "iennes","iste",         "istes","er", "ers","eron", "erons","eronne","trice","oir", "oire","oires", "oirs","ier", "iers","ière",         "ières","erie","eries","anderie","aire", "aires","ain", "aines", "ée","ées","aille", "ard","asse", "asses", "assier","âtre","aut","eau", "eaux","ceau", "ereau","eteau", "elle","elles", "et","elet","ets","ette","elette","ettes", "elettes","in", "ins","otin", "ine","ines", "illon","on","ons","ille", "erole","eroles", "ole","oles", "iche"]

    for phrase in corpus: # ajout des features additionnelles
        for prev, word, suiv in zip(["START"] + phrase["mots"][:-1], phrase["mots"], phrase["mots"][1:] + ["END"]):
            # création de triplets (mot précédent, mot, mot suivant)
            # avec "START" en prev pour le 1er mot
            # et "END" en suiv pour le dernier

            lemma = phrase["lemme"][phrase["mots"].index(word)] 

            # ajout d'un dictionnaire de features du mot au corpus
            corpus_features.append({mot : word, 
                gold : phrase["gold_labels"][phrase["mots"].index(word)], # on récupère le gold_label correspondant
                mot_prec : prev,
                mot_suiv : suiv,
                maj : word.istitle(),
                all_caps : word.isupper(),
                num : any(char.isdigit() for char in word), # mieux que isnumeric(), car renvoie false si espace (40 000) ou virgule (50,6) par ex
                nonAlphanum : not word.isalnum(),
                long : len(word) < 3,
                court : len(word) >= 3,
                un_car : len(word) == 1,
                suff_adv : word.endswith("ment"),
                suff_noun : any(word.endswith(elem) and len(word) != len(elem) for elem in list_noun),
                suff_adj : any(word.endswith(elem) and len(word) != len(elem) for elem in list_adj),
                suff_vb : any(word.endswith(elem) for elem in list_vb)
                # on vérifie la longueur du mot pour être sûr que ce soit un suffixe car on peut avoir le mot                      age avec le suffixe age par exemple ou bien aux
                # suff_noun : any(lemma.endswith(elem) and len(word) != len(elem) for elem in list_noun),
                # suff_adj : any(lemma.endswith(elem) for elem in list_adj),
                # suff_vb : any(lemma.endswith(elem) for elem in list_vb)
                
                })

    return corpus_features # renvoie les features transformés en vecteurs one-hot


gsd_train_features = feature_extraction(gsd_train, gsd_train_voc)

In [10]:
print(len(gsd_train_features))
print(*gsd_train_features[10:20], sep="\n")

364349
{'mot': "qu'", 'gold_label': 'SCONJ', 'préc': 'sport', 'suiv': 'on', 'maj': False, 'all caps': False, 'nb': False, 'non alphanum': True, 'long': False, 'court': True, 'un caractère': False, 'suffixe adv': False, 'suffixe nom': False, 'suffixe adj': False, 'suffixe verb': False}
{'mot': 'on', 'gold_label': 'PRON', 'préc': "qu'", 'suiv': 'les', 'maj': False, 'all caps': False, 'nb': False, 'non alphanum': False, 'long': True, 'court': False, 'un caractère': False, 'suffixe adv': False, 'suffixe nom': False, 'suffixe adj': False, 'suffixe verb': False}
{'mot': 'les', 'gold_label': 'PRON', 'préc': 'on', 'suiv': 'considére', 'maj': False, 'all caps': False, 'nb': False, 'non alphanum': False, 'long': False, 'court': True, 'un caractère': False, 'suffixe adv': False, 'suffixe nom': False, 'suffixe adj': False, 'suffixe verb': False}
{'mot': 'considére', 'gold_label': 'VERB', 'préc': 'les', 'suiv': 'presque', 'maj': False, 'all caps': False, 'nb': False, 'non alphanum': False, 'long': 

Conversion en vecteurs creux à l'aide de dictionnaires.

In [64]:
def make_sparse_vec(corpus_features, voc):
    taille_voc = len(voc)
    sparse_vecs = []
    for word in corpus_features:
        vec_word = {"mot":word[mot], "gold_label": word[gold], 'vec' : {}} # on associe le gold_label directement pour faciliter l'apprentissage, et le mot pour faciliter le débuggage
        marqueur = 0 # initialisation du marqueur pour indiquer où on en est du remplissage du vecteur
        for feature,value in word.items():
            if isinstance(value, str) and not feature == gold : # si la valeur est une chaîne et ce n'est
                vec_word["vec"][marqueur + voc.get(value.lower())] = 1 # on ajoute une entrée au dictionnaire avec comme clé l'indice dans la liste (+ le marqueur, puisqu'on avance dans le vecteur) et comme clé 1
                marqueur += taille_voc
            elif isinstance(value, int) or isinstance(value, bool):
                marqueur += 1
                if not int(value) == 0:
                    vec_word["vec"][marqueur] = int(value) # on ajoute la valeur dans la case suivante, convertie en entier si c'était un booléen
        sparse_vecs.append(vec_word)
    return sparse_vecs

gsd_train_vecs = make_sparse_vec(gsd_train_features, gsd_train_voc)

In [65]:
print("Aperçus des features avant et après vectorisation\n")
print(*zip(gsd_train_features[1034:1038] ,gsd_train_vecs[1034:1038]), sep="\n\n")

Aperçus des features avant et après vectorisation

({'mot': ',', 'gold_label': 'PUNCT', 'préc': 'Weinstein', 'suiv': 'qui', 'maj': False, 'all caps': False, 'nb': False, 'non alphanum': True, 'long': True, 'court': False, 'un caractère': True, 'suffixe adv': False, 'suffixe nom': False, 'suffixe adj': False, 'suffixe verb': False}, {'mot': ',', 'gold_label': 'PUNCT', 'vec': {32293: 1, 45385: 1, 79734: 1, 119128: 1, 119129: 1, 119131: 1}})

({'mot': 'qui', 'gold_label': 'PRON', 'préc': ',', 'suiv': 'a', 'maj': False, 'all caps': False, 'nb': False, 'non alphanum': False, 'long': False, 'court': True, 'un caractère': False, 'suffixe adv': False, 'suffixe nom': False, 'suffixe adj': False, 'suffixe verb': False}, {'mot': 'qui', 'gold_label': 'PRON', 'vec': {318: 1, 72001: 1, 101576: 1, 119130: 1}})

({'mot': 'a', 'gold_label': 'AUX', 'préc': 'qui', 'suiv': 'co-arrangé', 'maj': False, 'all caps': False, 'nb': False, 'non alphanum': False, 'long': True, 'court': False, 'un caractère': True,

## Implémentation de l'algorithme de classification
On a choisi d'implémenter la classification avec un perceptron moyenné.

Sources : cours 3/12 + perceptron.pdf + zoom du 17/12

La fonction predict sera utilisée à la fois dans l'apprentissage et dans la "prédiction". Elle correspond à la recherche de l'étiquette avec le plus grand score (argmax...)

In [40]:
def predict(word_features, weights):
    """Renvoie l'étiquette avec le plus gros score"""
    scores = {}
    for tag, w in weights.items():
        scores[tag] =  sum(word_features.get(feat)*w.get(feat, 0) for feat in word_features)
    return max(scores, key=scores.get)

In [76]:
from random import shuffle

def perceptron_train(training_set, voc, MAX_EPOCH=1): # ça serait une bonne idée de faire une classe pour éviter d'avoir à redonner le vocabulaire en argument à chaque fois par exemple
    
    tags = ["ADJ", "ADP", "ADV", "AUX", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "PUNCT", "SCONJ", "SYM", "VERB", "X"]

    # TODO initialisation des poids : à zéros ou bien de manière aléatoire ??

    # initialisation de a (poids totaux)
    a = {}
    for tag in tags:
        a[tag] = {}
    
    for i in range(0, MAX_EPOCH):

        # initialisation des vecteurs de poids
        w = {}
        for tag in tags:
            w[tag] = {}

        shuffled_set = training_set.copy() # copie du training set
        shuffle(shuffled_set) # mélange du training set

        for word in shuffled_set:
            prediction = predict(word['vec'], w)
            # print(word[mot], word[gold], prediction)
            if not word[gold] == "_" and not prediction == word[gold]: # si le gold_label n'est pas égal à celui prédit
                # on ignore les mots dont le gold_label est "_" ("au" et "du") car ils sont ensuite analysés comme (à le, de le)

                for feat in word['vec']:
                    w[word[gold]][feat] = w[word[gold]].get(feat,0) + word['vec'].get(feat) #  on ajoute x(i) à chaque poids de l'étiquette correcte
                    w[prediction][feat] = w[prediction].get(feat,0) - word['vec'][feat] #  on retire x(i) à chaque poids de l'étiquette mal prédite
           
        # on ajoute w à a
        for tag in w:
            for index in w[tag].keys(): 
                a[tag][index] = a[tag].get(index, 0) + w[tag][index]
    return a

poids_gsd_train = perceptron_train(gsd_train_vecs, gsd_train_voc)

## Evaluation des performances sur le corpus de validation (Dev)
Il faudra bien tester les hyper paramètres (epoch...)

## Evaluation sur le corpus Test

# Evaluation hors-domaine

## Analyse de l'impact du changement de domaine
* identifier causes de la baisse de performance : analyse de sortie, matrice de confusion, erreurs les + fréquentes
* réfléchir à des caractéristiques plus adaptées
* sélection d'un nouvel ensemble d'apprentissage avec des exemples représentatif (généré par un modèle de langue type TP2)

### Corpus UGC (User-generated content)

### Corpus littéraire

## Développement de systèmes robutes au changement de domaine